In [141]:
import itertools
import math

#total number of restaurants
NUM_RESTAURANTS = 10

#dictionary with restaurants 
#key: restaurant id
#value: coordinates
restaurants = {restaurant_id : (random.randint(0, 10), random.randint(0, 10)) for restaurant_id in range(NUM_RESTAURANTS)}

#compute the distance between two restaurants
#agrs:
#     point_a: coordinates of point a
#     point_b: coordinates of point b
#RETURN: distance between point a and point b
def compute_distance(point_a, point_b):
    x1,y1 = point_a
    x2,y2 = point_b

    return math.sqrt((x2 - x1)**2 + (y2 - y1)**2)  

print(restaurants)

{0: (10, 9), 1: (2, 9), 2: (7, 8), 3: (8, 3), 4: (6, 10), 5: (2, 6), 6: (10, 5), 7: (7, 0), 8: (5, 4), 9: (2, 3)}


In [170]:
#compute distances of all possible trips with two possible stops (not taking into account users yet)
#agrs:
#     request_list: dict with request to restaurants (no final user yet)
#RETURN: all restaurant combination and its costs (i.e. trips)
def compute_all_combination(request_dict, maximum_trips = 2):
    
    # list all restarurants for each client
    restaurantsc_id = ["{}*{}".format(key, request_dict[key]) for key in request_dict.keys()]
    
    # all possible trips with two stops
    # there are repeated trips i.e. (mc_donalds, kfc) and (kfc, mc_donals) becasue this will affect who should get the order
    combinations = [(x,y) for x,y in itertools.product(restaurantsc_id, repeat=maximum_trips) if x!=y]
    
    all_comb_dist = []

    for combination in combinations:
        rest_coord_1 = restaurants[int(combination[0].split("*")[1])]
        rest_coord_2 = restaurants[int(combination[1].split("*")[1])]

        distance = compute_distance(rest_coord_1,rest_coord_2)
        all_comb_dist.append((combination, distance))
    
    return all_comb_dist

NUM_REQUESTS = 10

#dictionary with requests
#request: request id
#value: restaurant
requests = {request_id : (random.randint(0, NUM_RESTAURANTS-1)) for request_id in range(NUM_REQUESTS)}

trips = compute_all_combination(requests)

print(trips)


[(('0*7', '1*5'), 7.810249675906654), (('0*7', '2*6'), 5.830951894845301), (('0*7', '3*8'), 4.47213595499958), (('0*7', '4*1'), 10.295630140987), (('0*7', '5*3'), 3.1622776601683795), (('0*7', '6*5'), 7.810249675906654), (('0*7', '7*8'), 4.47213595499958), (('0*7', '8*6'), 5.830951894845301), (('0*7', '9*9'), 5.830951894845301), (('1*5', '0*7'), 7.810249675906654), (('1*5', '2*6'), 8.06225774829855), (('1*5', '3*8'), 3.605551275463989), (('1*5', '4*1'), 3.0), (('1*5', '5*3'), 6.708203932499369), (('1*5', '6*5'), 0.0), (('1*5', '7*8'), 3.605551275463989), (('1*5', '8*6'), 8.06225774829855), (('1*5', '9*9'), 3.0), (('2*6', '0*7'), 5.830951894845301), (('2*6', '1*5'), 8.06225774829855), (('2*6', '3*8'), 5.0990195135927845), (('2*6', '4*1'), 8.94427190999916), (('2*6', '5*3'), 2.8284271247461903), (('2*6', '6*5'), 8.06225774829855), (('2*6', '7*8'), 5.0990195135927845), (('2*6', '8*6'), 0.0), (('2*6', '9*9'), 8.246211251235321), (('3*8', '0*7'), 4.47213595499958), (('3*8', '1*5'), 3.605551

In [171]:
from munkres import Munkres, print_matrix
from scipy.optimize import linear_sum_assignment

#use this example: http://software.clapper.org/munkres/

import random


NUM_DRIVERS = 3

#dictionary with delivery_driver
#key: driver id
#value: coordinates
drivers = {driver_id : (random.randint(0, 10), random.randint(0, 10)) for driver_id in range(NUM_DRIVERS)}


#create cost matrix where y_axis are drivers x_axis are trips
#agrs:
#     trips: all possible trips
#RETURN: a matrix (i.e. list of lists) [NUM_DRIVERS,NUM_TRIPS]
def create_matrix(drivers, trips):
    
    #drivers id
    drivers_ids = [driver for driver in drivers]
    
    #we assign an id to each trip by concatenating both restaurants
    trips_ids = ['%'.join(trip[0]) for trip in trips]   
    
    driver_restaruants_matrix = []
    for driver_id in drivers.keys():
        row = []
        for trip in trips:

            #get name of first restaurant of the trip (i.e. first stop)
            first_restaurant = trip[0][0].split("*")[0]
            
            #cost from driver to first restaruant
            cost_d2r = compute_distance(drivers[driver_id], restaurants[int(first_restaurant)])
            
            #cost all travel around all restaurants 
            cost_rall = trip[1]
            
            #TODO
            cost_final_clinet = 0
            
            cost_total = cost_d2r + cost_rall + cost_final_clinet
            
            row.append(cost_total)
            
        driver_restaruants_matrix.append(row)
    
    return drivers_ids, trips_ids, driver_restaruants_matrix

    
drivers_ids, trips_ids, cost_matrix = create_matrix(drivers, trips)

#optimize matrix (i.e. Hungarian algorithm)
m = Munkres()
indexes = m.compute(cost_matrix)


total=0
for row, column in indexes:
    value = cost_matrix[row][column]
    total += value
    #print(f'({row}, {column}) -> {value}')
    print("driver {} goes to {}".format(drivers_ids[row], trips_ids[column].split("%")))
    
print("all requests")
print(requests)

row_ind, col_ind = linear_sum_assignment(cost_matrix)

print(row_ind)
print(col_ind)


driver 0 goes to ['8*6', '2*6']
driver 1 goes to ['2*6', '8*6']
driver 2 goes to ['1*5', '6*5']
all requests
{0: 7, 1: 5, 2: 6, 3: 8, 4: 1, 5: 3, 6: 5, 7: 8, 8: 6, 9: 9}
[0 1 2]
[74 25 14]


In [ ]:
#PROBLEMS:
# the duplicate trick will not work, because the end point of the first worker will make it inconsistent
# the hack of multiple trips seems to be working, but we need to check a better solution eventually

In [83]:
#things TODO
# - take client into account (this is when we compute all taks with requests is the combination)
#     - we can increase the complexity by adding drops and picks combinations
#     - this can be done easily with itertools
# - try to do this offline to improve speed